
# Projeto 1 — Modelo de Concessão de Crédito (Classificação Binária)
**Autora:** Taísa Tavares Cruz  
**Atualizado em:** 2025-08-25 12:47

Este notebook entrega um **MVP funcional e legível** do pipeline de modelagem, mesmo que parcial, com:
- Entendimento da base e **EDA mínima guiada** (com checagens de shape, nulos, dtypes, TARGET).
- **Pré-processamento** (seleção de variáveis, imputação, encoding, scale quando necessário).
- **Treinamento de um baseline** de classificação (Logistic Regression) e comparação com um modelo de árvore (RandomForest).
- **Avaliação técnica** (ROC AUC, PR AUC, KS, matriz de confusão em um limiar sugerido).
- **Análise financeira (AS-IS vs TO-BE)** para **agosto/2017** conforme o enunciado.
- **Escoragem OOT** e salvamento do arquivo com `ID` e `score`.

> **Observação:** algumas etapas podem estar simplificadas por limitação de tempo. Em entrevista, explicarei as melhorias previstas (feature selection, tuning, calibração, explicabilidade, rejeição controlada etc.).


In [1]:

# ===== Imports e Configuração =====
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix, roc_curve, precision_recall_curve

pd.set_option('display.max_columns', 200)
RANDOM_STATE = 42

DATA_DIR = '../datasets/credit_01'
TRAIN_PATH = os.path.join(DATA_DIR, 'train.csv')
TEST_PATH  = os.path.join(DATA_DIR, 'test.csv')   # conforme enunciado, também é jan-ago/2017
OOT_PATH   = os.path.join(DATA_DIR, 'oot.csv')    # set/2017 a nov/2017

ID_COL = 'ID'           # ajustar se necessário
TARGET_COL = 'TARGET'   # 1 = Mau, 0 = Bom


In [2]:
import pandas as pd

path = r"C:\Users\taisa\Documents\monitoring\datasets\credit_01"

# Tente primeiro com ponto e vírgula
try:
    df_train = pd.read_csv(path + r"\train", sep=";", encoding="latin-1")
    print("Arquivo train carregado com sucesso!")
except Exception as e:
    print("Erro:", e)

# Conferindo as primeiras linhas
df_train.head()



Arquivo train carregado com sucesso!


,"REF_DATE,TARGET,VAR2,IDADE,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9,VAR10,VAR11,VAR12,VAR13,VAR14,VAR15,VAR16,VAR17,VAR18,VAR19,VAR20,VAR21,VAR22,VAR23,VAR24,VAR25,VAR26,VAR27,VAR28,VAR29,VAR30,VAR31,VAR32,VAR33,VAR34,VAR35,VAR36,VAR37,VAR38,VAR39,VAR40,VAR41,VAR42,VAR43,VAR44,VAR45,VAR46,VAR47,VAR48,VAR49,VAR50,VAR51,VAR52,VAR53,VAR54,VAR55,VAR56,VAR57,VAR58,VAR59,VAR60,VAR61,VAR62,VAR63,VAR64,VAR65,VAR66,VAR67,VAR68,VAR69,VAR70,VAR71,VAR72,VAR73,VAR74,VAR75,VAR76,VAR77,VAR78,VAR79,VAR80,VAR81,VAR82,VAR83,VAR84,VAR85,VAR86,VAR87,VAR88,VAR89,VAR90,VAR91,VAR92,VAR93,VAR94,VAR95,VAR96,VAR97,VAR98,VAR99,VAR100,VAR101,VAR102,VAR103,VAR104,VAR105,VAR106,VAR107,VAR108,VAR109,VAR110,VAR111,VAR112,VAR113,VAR114,VAR115,VAR116,VAR117,VAR118,VAR119,VAR120,VAR121,VAR122,VAR123,VAR124,VAR125,VAR126,VAR127,VAR128,VAR129,VAR130,VAR131,VAR132,VAR133,VAR134,VAR135,VAR136,VAR137,VAR138,VAR139,VAR140,VAR141,VAR142,VAR143,VAR144,VAR145,VAR146,VAR147,VAR148,VAR149,ID"
0,"2017-06-01 00:00:00+00:00,0,M,34.137,,RO,-8.80..."
1,"2017-08-18 00:00:00+00:00,0,M,40.447,,PB,-7.14..."
2,"2017-06-30 00:00:00+00:00,0,F,33.515,,RS,-27.9..."
3,"2017-08-05 00:00:00+00:00,1,F,25.797,,BA,-12.9..."
4,"2017-07-29 00:00:00+00:00,0,F,54.074,,RS,-30.0..."


In [3]:
import pandas as pd

# Caminho do arquivo
path_train = r"C:\Users\taisa\Documents\monitoring\datasets\credit_01\train"

# Carregando o arquivo
df_train = pd.read_csv(path_train, sep=",", encoding="latin-1")

# --- 1. Tamanho do dataset ---
print(f"Número de linhas: {df_train.shape[0]}")
print(f"Número de colunas: {df_train.shape[1]}")

# --- 2. Tipos de dados ---
print("\nTipos de dados:")
print(df_train.dtypes)

# --- 3. Primeiras linhas ---
print("\nPrimeiras 5 linhas:")
print(df_train.head())

# --- 4. Estatísticas descritivas ---
print("\nEstatísticas gerais (numéricas):")
print(df_train.describe())

# --- 5. Valores nulos por coluna ---
print("\nValores nulos por coluna:")
print(df_train.isnull().sum())

# --- 6. Balanceamento do TARGET ---
print("\nDistribuição do TARGET:")
print(df_train['TARGET'].value_counts())
print("\nPercentual do TARGET:")
print(df_train['TARGET'].value_counts(normalize=True)*100)

# --- 7. Estatísticas da idade ---
if 'IDADE' in df_train.columns:
    print("\nEstatísticas da idade:")
    print(df_train['IDADE'].describe())
else:
    print("\nColuna 'IDADE' não encontrada.")

# --- 8. Contagem de valores únicos em algumas colunas (opcional) ---
for col in ['VAR2', 'VAR4', 'VAR5']:
    if col in df_train.columns:
        print(f"\nValores únicos em {col}:")
        print(df_train[col].value_counts())


Número de linhas: 120750
Número de colunas: 151

Tipos de dados:
REF_DATE     object
TARGET        int64
VAR2         object
IDADE       float64
VAR4         object
             ...   
VAR146      float64
VAR147        int64
VAR148       object
VAR149       object
ID            int64
Length: 151, dtype: object

Primeiras 5 linhas:
                    REF_DATE  TARGET VAR2   IDADE VAR4 VAR5       VAR6  \
0  2017-06-01 00:00:00+00:00       0    M  34.137  NaN   RO  -8.808779   
1  2017-08-18 00:00:00+00:00       0    M  40.447  NaN   PB  -7.146537   
2  2017-06-30 00:00:00+00:00       0    F  33.515  NaN   RS -27.900178   
3  2017-08-05 00:00:00+00:00       1    F  25.797  NaN   BA -12.948874   
4  2017-07-29 00:00:00+00:00       0    F  54.074  NaN   RS -30.051810   

        VAR7 VAR8 VAR9      VAR10  VAR11  VAR12  VAR13  VAR14  VAR15  \
0 -63.878470    D    E      BAIXA    1.0  0.182    NaN  0.141    NaN   
1 -34.926080    E    E      MEDIA    0.0    NaN    NaN  0.136  0.127   
2 -53.

              TARGET          IDADE           VAR6           VAR7  \
count  120750.000000  107040.000000  117394.000000  117394.000000   
mean        0.245027      42.125255     -14.411389     -45.903480   
std         0.430105      15.198476       8.995077       7.529788   
min         0.000000      18.014000     -33.521563     -72.900276   
25%         0.000000      30.057250     -22.842778     -49.903564   
50%         0.000000      39.867000     -13.010590     -46.574908   
75%         0.000000      52.997000      -6.357067     -39.023621   
max         1.000000     105.477000       4.602823     -32.429516   

              VAR11         VAR12         VAR13         VAR14         VAR15  \
count  74488.000000  65724.000000  15530.000000  95197.000000  58269.000000   
mean       0.235917      0.290241      0.313850      0.241245      0.185754   
std        0.625609      0.308937      0.241229      0.262687      0.195320   
min       -4.000000      0.000000      0.000000      0.000000 

## 1) Carregamento dos Dados

In [4]:

import pandas as pd
import os

# Caminho base
base_path = r"C:\Users\taisa\Documents\monitoring\datasets\credit_01"

# Arquivos
train_path = os.path.join(base_path, "train")
test_path = os.path.join(base_path, "test")
oot_path = os.path.join(base_path, "oot")

# Carregando os datasets
df_train = pd.read_csv(train_path, sep=",", encoding="latin-1")
df_test = pd.read_csv(test_path, sep=",", encoding="latin-1")
df_oot = pd.read_csv(oot_path, sep=",", encoding="latin-1")

# Conferindo tamanhos
print(f"Train: {df_train.shape}")
print(f"Test: {df_test.shape}")
print(f"OOT: {df_oot.shape}")

# Conferindo primeiras linhas do treino
df_train.head()



Train: (120750, 151)
Test: (51751, 151)
OOT: (91965, 150)


,REF_DATE,TARGET,VAR2,IDADE,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9,VAR10,VAR11,VAR12,VAR13,VAR14,VAR15,VAR16,VAR17,VAR18,VAR19,VAR20,VAR21,VAR22,VAR23,VAR24,VAR25,VAR26,VAR27,VAR28,VAR29,VAR30,VAR31,VAR32,VAR33,VAR34,VAR35,VAR36,VAR37,VAR38,VAR39,VAR40,VAR41,VAR42,VAR43,VAR44,VAR45,VAR46,VAR47,VAR48,VAR49,VAR50,VAR51,VAR52,VAR53,VAR54,VAR55,VAR56,VAR57,VAR58,VAR59,VAR60,VAR61,VAR62,VAR63,VAR64,VAR65,VAR66,VAR67,VAR68,VAR69,VAR70,VAR71,VAR72,VAR73,VAR74,VAR75,VAR76,VAR77,VAR78,VAR79,VAR80,VAR81,VAR82,VAR83,VAR84,VAR85,VAR86,VAR87,VAR88,VAR89,VAR90,VAR91,VAR92,VAR93,VAR94,VAR95,VAR96,VAR97,VAR98,VAR99,VAR100,VAR101,VAR102,VAR103,VAR104,VAR105,VAR106,VAR107,VAR108,VAR109,VAR110,VAR111,VAR112,VAR113,VAR114,VAR115,VAR116,VAR117,VAR118,VAR119,VAR120,VAR121,VAR122,VAR123,VAR124,VAR125,VAR126,VAR127,VAR128,VAR129,VAR130,VAR131,VAR132,VAR133,VAR134,VAR135,VAR136,VAR137,VAR138,VAR139,VAR140,VAR141,VAR142,VAR143,VAR144,VAR145,VAR146,VAR147,VAR148,VAR149,ID
0,2017-06-01 00:00:00+00:00,0,M,34.137,NaN,RO,-8.808779,-63.878470,D,E,BAIXA,1.0,0.182,NaN,0.141,NaN,0.416667,NaN,1.263014,NaN,NaN,NaN,NaN,NaN,0.034,NaN,NaN,NaN,0.0,NaN,0.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.512334,0.486768,0.357526,0.538737,NaN,NaN,NaN,NaN,0.046,NaN,S,N,N,N,N,N,N,N,N,N,N,N,S,N,N,N,N,MEDIA,BAIXISSIMA,ALTA,BAIXISSIMA,ALTA,ALTISSIMA,BAIXISSIMA,BAIXISSIMA,ALTA,ALTA,ALTA,MEDIA,ALTA,ALTA,ALTA,ALTISSIMA,ALTA,MEDIA,ALTA,MEDIA,ALTA,ALTISSIMA,ALTISSIMA,MEDIA,BAIXISSIMA,ALTISSIMA,MEDIA,BAIXISSIMA,ALTA,BAIXISSIMA,ALTA,MEDIA,MEDIA,ALTA,BAIXISSIMA,ALTA,LONGE,LONGE,LONGE,PROXIMO,MEDIO,LONGE,MEDIO,LONGE,LONGE,LONGE,MEDIO,LONGE,PROXIMO,MEDIO,MEDIO,LONGE,PROXIMO,MEDIO,PROXIMO,MUITO LONGE,LONGE,PROXIMO,MEDIO,PROXIMO,LONGE,MUITO LONGE,MEDIO,LONGE,LONGE,MEDIO,LONGE,MUITO LONGE,LONGE,MEDIO,MUITO LONGE,LONGE,LONGE,MEDIO,LONGE,2680.289259,D,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE#@#NOME INEXISTENTE#@#CEP INE...,2.6.1,181755
1,2017-08-18 00:00:00+00:00,0,M,40.447,NaN,PB,-7.146537,-34.926080,E,E,MEDIA,0.0,NaN,NaN,0.136,0.127,NaN,NaN,0.654795,1.545205,NaN,NaN,0.125,NaN,0.017,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.328021,0.447454,0.414335,0.485512,NaN,NaN,NaN,NaN,0.099,NaN,S,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,ALTA,ALTISSIMA,ALTA,BAIXISSIMA,ALTA,ALTISSIMA,ALTA,MEDIA,ALTA,MEDIA,ALTISSIMA,MEDIA,MEDIA,ALTA,BAIXISSIMA,ALTA,ALTA,MEDIA,ALTISSIMA,ALTA,ALTA,ALTA,ALTA,MEDIA,BAIXISSIMA,ALTISSIMA,MEDIA,ALTA,ALTISSIMA,ALTA,ALTISSIMA,ALTA,ALTA,ALTISSIMA,ALTISSIMA,MEDIA,MEDIO,LONGE,MEDIO,MUITO LONGE,PROXIMO,MEDIO,PROXIMO,MEDIO,MEDIO,MEDIO,LONGE,LONGE,LONGE,LONGE,PROXIMO,LONGE,PROXIMO,PROXIMO,MUITO LONGE,MEDIO,MEDIO,PROXIMO,PROXIMO,PROXIMO,LONGE,MEDIO,PROXIMO,LONGE,MEDIO,MEDIO,MEDIO,LONGE,MEDIO,PROXIMO,PROXIMO,LONGE,LONGE,LONGE,MUITO LONGE,1777.725469,E,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE#@#NOME INEXISTENTE#@#CEP INE...,2.6.1,287633
2,2017-06-30 00:00:00+00:00,0,F,33.515,NaN,RS,-27.900178,-53.314035,NaN,E,ALTISSIMA,NaN,0.095,NaN,0.152,NaN,0.166667,NaN,1.665753,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,124.0,NaN,0.627262,0.730539,0.916771,0.519726,NaN,NaN,NaN,NaN,1.000,NaN,S,N,N,N,N,N,N,S,N,N,N,N,N,N,S,N,N,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,ALTA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,MEDIA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,MEDIA,LONGE,MUITO LONGE,LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,LONGE,LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,LONGE,LONGE,MUITO LONGE,LONGE,MUITO LONGE,LONGE,PROXIMO,MUITO LONGE,LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,LONGE,MUITO LONGE,LONGE,LONGE,MUITO LONGE,LONGE,MUITO PROXIMO,1695.494979,E,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE#@#NOME INEXISTENTE#@#CEP INE...,2.6.1,88015
3,2017-08-05 00:00:00

## 2) EDA Essencial

In [5]:

# --- EDA Essencial do train ---
print("### EDA ESSENCIAL - TRAIN ###\n")

# 1. Tamanho do dataset
print(f"Número de linhas: {df_train.shape[0]}")
print(f"Número de colunas: {df_train.shape[1]}\n")

# 2. Tipos de dados
print("Tipos de dados:")
print(df_train.dtypes, "\n")

# 3. Primeiras linhas
print("Primeiras 5 linhas:")
display(df_train.head())

# 4. Estatísticas descritivas
print("\nEstatísticas gerais (numéricas):")
display(df_train.describe())

# 5. Valores nulos por coluna
print("\nValores nulos por coluna:")
display(df_train.isnull().sum())

# 6. Balanceamento do TARGET
print("\nDistribuição do TARGET:")
print(df_train['TARGET'].value_counts())
print("\nPercentual do TARGET:")
print(df_train['TARGET'].value_counts(normalize=True)*100)

# 7. Estatísticas da idade
if 'IDADE' in df_train.columns:
    print("\nEstatísticas da idade:")
    display(df_train['IDADE'].describe())
else:
    print("\nColuna 'IDADE' não encontrada.")

# 8. Contagem de valores únicos em algumas colunas categóricas (opcional)
for col in ['VAR2', 'VAR4', 'VAR5']:
    if col in df_train.columns:
        print(f"\nValores únicos em {col}:")
        display(df_train[col].value_counts())


### EDA ESSENCIAL - TRAIN ###

Número de linhas: 120750
Número de colunas: 151

Tipos de dados:
REF_DATE     object
TARGET        int64
VAR2         object
IDADE       float64
VAR4         object
             ...   
VAR146      float64
VAR147        int64
VAR148       object
VAR149       object
ID            int64
Length: 151, dtype: object 

Primeiras 5 linhas:


,REF_DATE,TARGET,VAR2,IDADE,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9,VAR10,VAR11,VAR12,VAR13,VAR14,VAR15,VAR16,VAR17,VAR18,VAR19,VAR20,VAR21,VAR22,VAR23,VAR24,VAR25,VAR26,VAR27,VAR28,VAR29,VAR30,VAR31,VAR32,VAR33,VAR34,VAR35,VAR36,VAR37,VAR38,VAR39,VAR40,VAR41,VAR42,VAR43,VAR44,VAR45,VAR46,VAR47,VAR48,VAR49,VAR50,VAR51,VAR52,VAR53,VAR54,VAR55,VAR56,VAR57,VAR58,VAR59,VAR60,VAR61,VAR62,VAR63,VAR64,VAR65,VAR66,VAR67,VAR68,VAR69,VAR70,VAR71,VAR72,VAR73,VAR74,VAR75,VAR76,VAR77,VAR78,VAR79,VAR80,VAR81,VAR82,VAR83,VAR84,VAR85,VAR86,VAR87,VAR88,VAR89,VAR90,VAR91,VAR92,VAR93,VAR94,VAR95,VAR96,VAR97,VAR98,VAR99,VAR100,VAR101,VAR102,VAR103,VAR104,VAR105,VAR106,VAR107,VAR108,VAR109,VAR110,VAR111,VAR112,VAR113,VAR114,VAR115,VAR116,VAR117,VAR118,VAR119,VAR120,VAR121,VAR122,VAR123,VAR124,VAR125,VAR126,VAR127,VAR128,VAR129,VAR130,VAR131,VAR132,VAR133,VAR134,VAR135,VAR136,VAR137,VAR138,VAR139,VAR140,VAR141,VAR142,VAR143,VAR144,VAR145,VAR146,VAR147,VAR148,VAR149,ID
0,2017-06-01 00:00:00+00:00,0,M,34.137,NaN,RO,-8.808779,-63.878470,D,E,BAIXA,1.0,0.182,NaN,0.141,NaN,0.416667,NaN,1.263014,NaN,NaN,NaN,NaN,NaN,0.034,NaN,NaN,NaN,0.0,NaN,0.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.512334,0.486768,0.357526,0.538737,NaN,NaN,NaN,NaN,0.046,NaN,S,N,N,N,N,N,N,N,N,N,N,N,S,N,N,N,N,MEDIA,BAIXISSIMA,ALTA,BAIXISSIMA,ALTA,ALTISSIMA,BAIXISSIMA,BAIXISSIMA,ALTA,ALTA,ALTA,MEDIA,ALTA,ALTA,ALTA,ALTISSIMA,ALTA,MEDIA,ALTA,MEDIA,ALTA,ALTISSIMA,ALTISSIMA,MEDIA,BAIXISSIMA,ALTISSIMA,MEDIA,BAIXISSIMA,ALTA,BAIXISSIMA,ALTA,MEDIA,MEDIA,ALTA,BAIXISSIMA,ALTA,LONGE,LONGE,LONGE,PROXIMO,MEDIO,LONGE,MEDIO,LONGE,LONGE,LONGE,MEDIO,LONGE,PROXIMO,MEDIO,MEDIO,LONGE,PROXIMO,MEDIO,PROXIMO,MUITO LONGE,LONGE,PROXIMO,MEDIO,PROXIMO,LONGE,MUITO LONGE,MEDIO,LONGE,LONGE,MEDIO,LONGE,MUITO LONGE,LONGE,MEDIO,MUITO LONGE,LONGE,LONGE,MEDIO,LONGE,2680.289259,D,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE#@#NOME INEXISTENTE#@#CEP INE...,2.6.1,181755
1,2017-08-18 00:00:00+00:00,0,M,40.447,NaN,PB,-7.146537,-34.926080,E,E,MEDIA,0.0,NaN,NaN,0.136,0.127,NaN,NaN,0.654795,1.545205,NaN,NaN,0.125,NaN,0.017,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.328021,0.447454,0.414335,0.485512,NaN,NaN,NaN,NaN,0.099,NaN,S,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,ALTA,ALTISSIMA,ALTA,BAIXISSIMA,ALTA,ALTISSIMA,ALTA,MEDIA,ALTA,MEDIA,ALTISSIMA,MEDIA,MEDIA,ALTA,BAIXISSIMA,ALTA,ALTA,MEDIA,ALTISSIMA,ALTA,ALTA,ALTA,ALTA,MEDIA,BAIXISSIMA,ALTISSIMA,MEDIA,ALTA,ALTISSIMA,ALTA,ALTISSIMA,ALTA,ALTA,ALTISSIMA,ALTISSIMA,MEDIA,MEDIO,LONGE,MEDIO,MUITO LONGE,PROXIMO,MEDIO,PROXIMO,MEDIO,MEDIO,MEDIO,LONGE,LONGE,LONGE,LONGE,PROXIMO,LONGE,PROXIMO,PROXIMO,MUITO LONGE,MEDIO,MEDIO,PROXIMO,PROXIMO,PROXIMO,LONGE,MEDIO,PROXIMO,LONGE,MEDIO,MEDIO,MEDIO,LONGE,MEDIO,PROXIMO,PROXIMO,LONGE,LONGE,LONGE,MUITO LONGE,1777.725469,E,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE#@#NOME INEXISTENTE#@#CEP INE...,2.6.1,287633
2,2017-06-30 00:00:00+00:00,0,F,33.515,NaN,RS,-27.900178,-53.314035,NaN,E,ALTISSIMA,NaN,0.095,NaN,0.152,NaN,0.166667,NaN,1.665753,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,124.0,NaN,0.627262,0.730539,0.916771,0.519726,NaN,NaN,NaN,NaN,1.000,NaN,S,N,N,N,N,N,N,S,N,N,N,N,N,N,S,N,N,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,ALTA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,MEDIA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,MEDIA,LONGE,MUITO LONGE,LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,LONGE,LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,LONGE,LONGE,MUITO LONGE,LONGE,MUITO LONGE,LONGE,PROXIMO,MUITO LONGE,LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,LONGE,MUITO LONGE,LONGE,LONGE,MUITO LONGE,LONGE,MUITO PROXIMO,1695.494979,E,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE#@#NOME INEXISTENTE#@#CEP INE...,2.6.1,88015
3,2017-08-05 00:00:00


Estatísticas gerais (numéricas):


,TARGET,IDADE,VAR6,VAR7,VAR11,VAR12,VAR13,VAR14,VAR15,VAR16,VAR17,VAR18,VAR19,VAR20,VAR21,VAR22,VAR23,VAR24,VAR25,VAR26,VAR27,VAR28,VAR29,VAR30,VAR37,VAR39,VAR40,VAR41,VAR42,VAR44,VAR46,VAR47,VAR141,VAR145,VAR146,VAR147,ID
count,120750.000000,107040.000000,117394.000000,117394.000000,74488.000000,65724.000000,15530.000000,95197.000000,58269.000000,44981.000000,15530.000000,95197.000000,58269.000000,11353.000000,15593.000000,58329.000000,15530.000000,97285.000000,58269.000000,1008.000000,1042.000000,12719.000000,24724.000000,12719.000000,25044.000000,120679.000000,117466.000000,117811.000000,107048.000000,401.000000,294.000000,120750.000000,120750.000000,679.000000,168.000000,120750.000000,120750.000000
mean,0.245027,42.125255,-14.411389,-45.903480,0.235917,0.290241,0.313850,0.241245,0.185754,2.044374,2.132795,0.978085,1.244427,0.045492,0.349243,0.212624,0.087496,0.080060,0.061524,0.359200,0.018753,0.003905,0.145150,0.194422,150.346550,0.461353,0.531287,0.631175,0.504619,1691.738429,6984.218469,0.256543,1854.833006,4018.743785,1942.649762,101.841656,165324.864199
std,0.430105,15.198476,8.995077,7.529788,0.625609,0.308937,0.241229,0.262687,0.195320,3.170869,2.171140,0.496078,1.121771,0.123168,0.192981,0.176746,0.127580,0.139072,0.125769,0.230197,0.034943,0.037876,0.149858,0.161558,82.413855,0.144093,0.115968,0.222607,0.069614,2177.830516,4621.964093,0.406746,893.999792,3700.836248,3143.757850,0.540016,95488.442320
min,0.000000,18.014000,-33.521563,-72.900276,-4.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.002740,0.002740,0.002740,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.000000,0.049309,0.208688,0.000000,0.006658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,3.000000
25%,0.000000,30.057250,-22.842778,-49.903564,0.000000,0.055000,0.056000,0.105000,0.078000,0.250000,0.493151,0.503899,0.347735,0.000000,0.250000,0.125000,0.011000,0.000000,0.000000,0.167000,0.000000,0.000000,0.042000,0.125000,102.000000,0.363787,0.447559,0.440572,0.463579,642.000000,3888.995000,0.000000,1513.227400,1633.195000,0.000000,102.000000,82727.250000
50%,0.000000,39.867000,-13.010590,-46.574908,0.000000,0.159000,0.318000,0.140000,0.112000,0.833333,0.816438,1.052055,1.142466,0.011000,0.250000,0.125000,0.043000,0.034000,0.000000,0.333000,0.000000,0.000000,0.104000,0.125000,134.000000,0.462267,0.510023,0.542298,0.496036,769.000000,5386.310000,0.003000,1627.157652,3024.480000,935.120000,102.000000,165298.000000
75%,0.000000,52.997000,-6.357067,-39.023621,0.000000,0.505000,0.507000,0.168000,0.153000,2.416667,4.079452,1.394521,1.463014,0.033000,0.500000,0.250000,0.096000,0.086000,0.065000,0.500000,0.033000,0.000000,0.188000,0.250000,182.000000,0.558495,0.582701,0.898052,0.522576,1747.500000,9601.390000,0.421000,1820.670284,5217.670000,2260.125000,102.000000,248248.000000
max,1.000000,105.477000,4.602823,-32.429516,4.000000,1.500000,1.000000,1.700000,1.000000,15.999999,8.999999,2.471023,7.406250,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.367000,1.000000,1.000000,1.000000,902.000000,0.953539,0.920685,1.000000,0.910865,17374.940000,26523.920000,1.000000,33954.140000,33954.140000,17229.200000,102.000000,330581.000000



Valores nulos por coluna:


...

F    60131
M    46000
Name: VAR2, dtype: int64


Valores únicos em VAR4:


S    202
Name: VAR4, dtype: int64


Valores únicos em VAR5:


SP    19079
BA    10306
PA    10159
RS     8410
CE     8262
MG     7757
PE     7056
RJ     4617
AM     4145
RN     3800
PR     3668
PB     3085
AL     2800
ES     2765
MS     2515
GO     2369
AC     2304
MA     2083
MT     2015
SC     2013
PI     1858
RO     1731
AP     1292
SE     1175
DF      790
TO      767
RR      573
Name: VAR5, dtype: int64

## 3) Pré-processamento

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# -------------------------
# 0. Variáveis alvo e ID
TARGET_COL = 'TARGET'
ID_COL = 'ID'

# -------------------------
# 1. Separar colunas numéricas e categóricas
feature_cols = [c for c in df_train.columns if c not in [ID_COL, TARGET_COL]]
num_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(df_train[c])]
cat_cols = [c for c in feature_cols if c not in num_cols]

print(f"Número de features: {len(feature_cols)} | Numéricas: {len(num_cols)} | Categóricas: {len(cat_cols)}")

# -------------------------
# 2. Transformar datas em variáveis numéricas, se existir REF_DATE
if 'REF_DATE' in df_train.columns:
    for df in [df_train, df_test, df_oot]:
        df['ANO'] = pd.to_datetime(df['REF_DATE']).dt.year
        df['MES'] = pd.to_datetime(df['REF_DATE']).dt.month
        df['DIA'] = pd.to_datetime(df['REF_DATE']).dt.day
    num_cols += ['ANO', 'MES', 'DIA']
    cat_cols = [c for c in cat_cols if c != 'REF_DATE']

# -------------------------
# 3. Label Encoding das categóricas (forma vetorizada, rápida)
le_dict = {}
for c in cat_cols:
    le = LabelEncoder()
    df_train[c] = le.fit_transform(df_train[c].astype(str))
    
    # Mapear teste e OOT
    le_map = dict(zip(le.classes_, le.transform(le.classes_)))
    df_test[c] = df_test[c].astype(str).map(le_map).fillna(-1).astype(int)
    df_oot[c]  = df_oot[c].astype(str).map(le_map).fillna(-1).astype(int)
    
    le_dict[c] = le

# -------------------------
# 4. Dividir train em train + validação
RANDOM_STATE = 42
X = df_train[feature_cols]
y = df_train[TARGET_COL]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, 
                                                  random_state=RANDOM_STATE, 
                                                  stratify=y)

print("Shape X_train:", X_train.shape)
print("Shape X_val:", X_val.shape)



Número de features: 149 | Numéricas: 35 | Categóricas: 114
Shape X_train: (96600, 149)
Shape X_val: (24150, 149)


## 4) Partição de Validação (dentro de `train`)

In [7]:

from sklearn.model_selection import train_test_split

# --- 4) Partição de validação dentro do treino ---
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, 
    test_size=0.2,       # 20% para validação
    random_state=42,     # garante reprodutibilidade
    stratify=y_train     # mantém proporção de classes TARGET
)

print(f"Tamanho treino: {X_tr.shape}, Tamanho validação: {X_val.shape}")


Tamanho treino: (77280, 149), Tamanho validação: (19320, 149)


## 5) Modelos Baseline

In [8]:

from sklearn.model_selection import train_test_split

# Definir proporção de validação (ex: 20%)
VAL_SIZE = 0.2
RANDOM_STATE = 42

df_tr, df_val = train_test_split(df_train, test_size=VAL_SIZE, random_state=RANDOM_STATE, stratify=df_train[TARGET_COL])

# Agora criar X e y para treino e validação
feature_cols = [c for c in df_train.columns if c not in [ID_COL, TARGET_COL]]
num_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(df_train[c])]
cat_cols = [c for c in feature_cols if c not in num_cols]

X_train = df_tr[feature_cols].copy()
y_train = df_tr[TARGET_COL].copy()

X_val = df_val[feature_cols].copy()
y_val = df_val[TARGET_COL].copy()



In [9]:
# Garantir que num_cols só tenha colunas numéricas
num_cols = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("Colunas numéricas:", num_cols)


Colunas numéricas: ['IDADE', 'VAR6', 'VAR7', 'VAR11', 'VAR12', 'VAR13', 'VAR14', 'VAR15', 'VAR16', 'VAR17', 'VAR18', 'VAR19', 'VAR20', 'VAR21', 'VAR22', 'VAR23', 'VAR24', 'VAR25', 'VAR26', 'VAR27', 'VAR28', 'VAR29', 'VAR30', 'VAR37', 'VAR39', 'VAR40', 'VAR41', 'VAR42', 'VAR44', 'VAR46', 'VAR47', 'VAR141', 'VAR145', 'VAR146', 'VAR147', 'ANO', 'MES', 'DIA']


In [10]:
# Detectar colunas binárias com 'S'/'N'
for col in num_cols:
    uniques = X_train[col].dropna().unique()
    # Converter todos os valores para string antes de verificar
    uniques_str = set([str(u).upper() for u in uniques])
    if uniques_str.issubset({'S','N'}):
        print(f"⚠️ Coluna {col} é binária S/N e será convertida para 0/1")
        # Converter para 0/1
        X_train[col] = X_train[col].map(lambda x: 1 if str(x).upper()=='S' else 0 if str(x).upper()=='N' else np.nan)
        X_val[col]   = X_val[col].map(lambda x: 1 if str(x).upper()=='S' else 0 if str(x).upper()=='N' else np.nan)
        df_test[col] = df_test[col].map(lambda x: 1 if str(x).upper()=='S' else 0 if str(x).upper()=='N' else np.nan)
        df_oot[col]  = df_oot[col].map(lambda x: 1 if str(x).upper()=='S' else 0 if str(x).upper()=='N' else np.nan)




In [11]:
# Forçar todas as colunas numéricas a float
for df in [X_train, X_val, df_test, df_oot]:
    for col in num_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # valores não numéricos viram NaN

# Agora sim aplicar imputação
imputer_num = SimpleImputer(strategy='median')

X_train[num_cols] = imputer_num.fit_transform(X_train[num_cols])
X_val[num_cols]   = imputer_num.transform(X_val[num_cols])
df_test[num_cols] = imputer_num.transform(df_test[num_cols])
df_oot[num_cols]  = imputer_num.transform(df_oot[num_cols])




## 6) Treino no conjunto completo e avaliação na base de Teste

In [15]:
# Lista de colunas em formato de lista
list(df_train.columns)


['REF_DATE',
 'TARGET',
 'VAR2',
 'IDADE',
 'VAR4',
 'VAR5',
 'VAR6',
 'VAR7',
 'VAR8',
 'VAR9',
 'VAR10',
 'VAR11',
 'VAR12',
 'VAR13',
 'VAR14',
 'VAR15',
 'VAR16',
 'VAR17',
 'VAR18',
 'VAR19',
 'VAR20',
 'VAR21',
 'VAR22',
 'VAR23',
 'VAR24',
 'VAR25',
 'VAR26',
 'VAR27',
 'VAR28',
 'VAR29',
 'VAR30',
 'VAR31',
 'VAR32',
 'VAR33',
 'VAR34',
 'VAR35',
 'VAR36',
 'VAR37',
 'VAR38',
 'VAR39',
 'VAR40',
 'VAR41',
 'VAR42',
 'VAR43',
 'VAR44',
 'VAR45',
 'VAR46',
 'VAR47',
 'VAR48',
 'VAR49',
 'VAR50',
 'VAR51',
 'VAR52',
 'VAR53',
 'VAR54',
 'VAR55',
 'VAR56',
 'VAR57',
 'VAR58',
 'VAR59',
 'VAR60',
 'VAR61',
 'VAR62',
 'VAR63',
 'VAR64',
 'VAR65',
 'VAR66',
 'VAR67',
 'VAR68',
 'VAR69',
 'VAR70',
 'VAR71',
 'VAR72',
 'VAR73',
 'VAR74',
 'VAR75',
 'VAR76',
 'VAR77',
 'VAR78',
 'VAR79',
 'VAR80',
 'VAR81',
 'VAR82',
 'VAR83',
 'VAR84',
 'VAR85',
 'VAR86',
 'VAR87',
 'VAR88',
 'VAR89',
 'VAR90',
 'VAR91',
 'VAR92',
 'VAR93',
 'VAR94',
 'VAR95',
 'VAR96',
 'VAR97',
 'VAR98',
 'VAR99',
 'V

In [17]:
# Converter REF_DATE para datetime
for df in [df_train, df_test, df_oot]:
    df['REF_DATE'] = pd.to_datetime(df['REF_DATE'], errors='coerce')


In [19]:
for df in [df_train, df_test, df_oot]:
    df['ANO'] = df['REF_DATE'].dt.year
    df['MES'] = df['REF_DATE'].dt.month
    df['DIA'] = df['REF_DATE'].dt.day


In [20]:
feature_cols = [c for c in df_train.columns if c not in ['ID', 'TARGET', 'REF_DATE']]


In [21]:
best_pipe.fit(df_train[feature_cols], df_train[TARGET_COL])
p_test_mau = best_pipe.predict_proba(df_test[feature_cols])[:,1]
score_test_bom = 1 - p_test_mau

from sklearn.metrics import roc_auc_score, average_precision_score
roc_test = roc_auc_score(df_test[TARGET_COL], p_test_mau)
pr_test  = average_precision_score(df_test[TARGET_COL], p_test_mau)
print(f"Teste - ROC AUC: {roc_test:.4f} | PR AUC: {pr_test:.4f}")


Teste - ROC AUC: 0.6804 | PR AUC: 0.3941


## 7) Análise Financeira (Agosto/2017) — AS-IS vs TO-BE

In [23]:

# --------------------------
# Passo 7 — Análise Financeira (Agosto/2017) — AS-IS vs TO-BE
# --------------------------

import numpy as np

# Criar máscara para agosto/2017
mask_agosto2017 = (df_test['ANO']==2017) & (df_test['MES']==8)

# Subset do DataFrame e do score correspondente
df_fin = df_test[mask_agosto2017].copy()
score_fin = score_test_bom[mask_agosto2017.values]  # alinhamento de tamanho

# Detectar coluna de idade
POSSIBLE_AGE_COLS = ['AGE', 'Idade', 'IDADE', 'age', 'idade']
age_col = None
for c in POSSIBLE_AGE_COLS:
    if c in df_test.columns:
        age_col = c
        break
if age_col is None:
    raise ValueError("⚠️ Coluna de Idade não encontrada automaticamente. Ajuste a variável 'age_col' manualmente.")
else:
    print(f"Coluna de idade detectada: {age_col}")

# Parâmetros
EMPRESTIMO = 1000.0  # R$

# Política AS-IS
def politica_as_is(df):
    negar = df[age_col] <= 28
    aprovar = ~negar

    carteira = aprovar.sum() * EMPRESTIMO
    inadimplentes = df.loc[aprovar, TARGET_COL].sum()  # total de Maus aprovados
    divida_total = inadimplentes * EMPRESTIMO
    perc_negados = negar.mean()

    return {
        'carteira_aprovada': carteira,
        'divida_total': divida_total,
        'perc_negados': perc_negados,
        'mask_aprovados': aprovar.values
    }

# Política TO-BE (mesma taxa de negados)
def politica_to_be_mesma_taxa_negados(df, score_bom):
    asis = politica_as_is(df)
    perc_negados = asis['perc_negados']

    thr = np.quantile(score_bom, perc_negados)  # cortar os scores mais baixos
    negar = score_bom < thr
    aprovar = ~negar

    carteira = aprovar.sum() * EMPRESTIMO
    inadimplentes = df.loc[aprovar, TARGET_COL].sum()
    divida_total = inadimplentes * EMPRESTIMO

    return {
        'carteira_aprovada': carteira,
        'divida_total': divida_total,
        'perc_negados': perc_negados,
        'threshold_score_bom': float(thr)
    }

# Executar análise
asis = politica_as_is(df_fin)
tobe = politica_to_be_mesma_taxa_negados(df_fin, score_fin)

economia = asis['divida_total'] - tobe['divida_total']

# Resultados
print("=== Política AS-IS (idade <= 28 nega) ===")
print(f"Carteira aprovada: R${asis['carteira_aprovada']:.2f}")
print(f"Dívida total: R${asis['divida_total']:.2f}")
print(f"% Negados: {asis['perc_negados']*100:.2f}%")

print("\n=== Política TO-BE (corte no SCORE Bom) ===")
print(f"Threshold SCORE Bom: {tobe['threshold_score_bom']:.4f}")
print(f"Carteira aprovada: R${tobe['carteira_aprovada']:.2f}")
print(f"Dívida total: R${tobe['divida_total']:.2f}")
print(f"% Negados (igual ao AS-IS): {tobe['perc_negados']*100:.2f}%")

print("\n=== Economia estimada (AS-IS → TO-BE) ===")
print(f"Economia: R${economia:.2f}")



Coluna de idade detectada: IDADE
=== Política AS-IS (idade <= 28 nega) ===
Carteira aprovada: R$5933000.00
Dívida total: R$1022000.00
% Negados: 18.78%

=== Política TO-BE (corte no SCORE Bom) ===
Threshold SCORE Bom: 0.7543
Carteira aprovada: R$5933000.00
Dívida total: R$916000.00
% Negados (igual ao AS-IS): 18.78%

=== Economia estimada (AS-IS → TO-BE) ===
Economia: R$106000.00


## 8) Limiar sugerido e Matriz de Confusão (Teste)

In [24]:

# Usando os mesmos dados do Passo 7
thr_bom = np.quantile(score_fin, (df_fin.shape[0] - df_fin.shape[0] * asis['perc_negados'])/df_fin.shape[0])
y_pred_negado = (score_fin < thr_bom)
aprovados = ~y_pred_negado

TP_mau_negado = ((y_pred_negado) & (df_fin[TARGET_COL]==1)).sum()
FN_mau_aprovado = ((aprovados) & (df_fin[TARGET_COL]==1)).sum()
TN_bom_aprovado = ((aprovados) & (df_fin[TARGET_COL]==0)).sum()
FP_bom_negado = ((y_pred_negado) & (df_fin[TARGET_COL]==0)).sum()

print("Contagens úteis sob o limiar TO-BE (Agosto/2017):")
print("Maus negados (TP):", TP_mau_negado)
print("Maus aprovados (FN):", FN_mau_aprovado)
print("Bons aprovados (TN):", TN_bom_aprovado)
print("Bons negados (FP):", FP_bom_negado)


Contagens úteis sob o limiar TO-BE (Agosto/2017):
Maus negados (TP): 1237
Maus aprovados (FN): 116
Bons aprovados (TN): 1256
Bons negados (FP): 4696


## 9) Escoragem da base OOT (Out-of-time) e Exportação

In [26]:

ID_COL = 'ID'  # ajuste se for diferente
oot_scores = best_pipe.predict_proba(df_oot[feature_cols])[:,1]
oot_score_bom = 1 - oot_scores

df_oot_score = pd.DataFrame({
    ID_COL: df_oot[ID_COL],
    'score_bom': oot_score_bom
})
OUT_PATH = 'C:/Users/taisa/Documents/monitoring/oot_scores.csv'
df_oot_score.to_csv(OUT_PATH, index=False)
print(f"Arquivo de scores OOT salvo em: {OUT_PATH}")




Arquivo de scores OOT salvo em: C:/Users/taisa/Documents/monitoring/oot_scores.csv
